# Introduzione all'Intelligenza Artificiale<br/>Supporto per le Esercitazioni


Questo notebook rappresenta una guida all'implementazione e utilizzo dei principali algoritmi e modelli presentati nell'ambito del corso "Introduzione all'Intelligenza Artificiale", corso di Laurea in Informatica, Università di Pisa, anno accademico 2024/2025.

Il codice presentato di seguito è un'implementazione delle funzioni riportate nel testo di riferimento del corso:<br/>
*S. Russell, P. Norvig, "Artificial Intelligence: A Modern Approach, Pearson, 4th Edition, 2020*

Ulteriori risorse possono essere reperite online all'indirizzo: https://github.com/aimacode

# Algoritmi di Ricerca

Di seguito, verrano presentati i principali algoritmi per risolvere problemi di ricerca.

## Classi e Strutture Dati
Introduciamo le astrazioni necessarie per poter rappresentare e risolvere un problema di ricerca.<br/>
Queste classi saranno i *building blocks* per poter successivamente implementare gli algoritmi risolutori.

### Come rappresentare un problema - La classe Problem